In [ ]:
import os
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import numpy as np

## Анализ Основного датасета

In [ ]:

# Путь к папке с данными
data_path = "../data/raw/Task 21"

# Список всех файлов в папке
file_list = os.listdir(data_path)

# Загрузка данных для одной скважины (пример)
def load_well_data(file_path):
    df = pd.read_csv(file_path, sep="\t", header=None, names=["Time (hours)", "Pressure (atm)"])
    return df

# Пример загрузки данных для первой скважины
well_id = file_list[0]
well_data = load_well_data(os.path.join(data_path, well_id))
print(well_data.head())

In [ ]:


# Выбор случайных 100 файлов
sample_files = random.sample(file_list, 100)

# Загрузка данных для выборочных файлов
sample_data = []
for well_id in sample_files:
    well_data = load_well_data(os.path.join(data_path, well_id))
    sample_data.append(well_data)

In [ ]:
def load_data_generator(file_list, data_path):
    for well_id in file_list:
        yield load_well_data(os.path.join(data_path, well_id))

# Пример использования генератора
data_gen = load_data_generator(file_list, data_path)

In [ ]:

def plot_pressure(data, well_id):
    plt.figure(figsize=(12, 6))
    plt.plot(data["Time (hours)"], data["Pressure (atm)"], label=f"Well {well_id}")
    plt.xlabel("Time (hours)")
    plt.ylabel("Pressure (atm)")
    plt.title(f"Pressure over Time for Well {well_id}")
    plt.legend()
    plt.grid()
    plt.show()

# Пример визуализации для первой скважины
plot_pressure(well_data, well_id)

In [ ]:
# Выбор 5 случайных скважин
random_wells = random.sample(file_list, 5)

# Построение графиков
for well_id in random_wells:
    well_data = load_well_data(os.path.join(data_path, well_id))
    plot_pressure(well_data, well_id)

In [ ]:
# Сбор данных по давлению для выборочных скважин
sample_pressures = [well["Pressure (atm)"].values for well in sample_data]
sample_pressures = np.concatenate(sample_pressures)

# Построение гистограммы
plt.figure(figsize=(10, 6))
sns.histplot(sample_pressures, bins=50, kde=True)
plt.title("Distribution of Pressure (Sample of 100 Wells)")
plt.xlabel("Pressure (atm)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# Расчет статистик для всех скважин
pressure_stats = []
for well_id in file_list:
    well_data = load_well_data(os.path.join(data_path, well_id))
    pressure_stats.append({
        "mean": well_data["Pressure (atm)"].mean(),
        "median": well_data["Pressure (atm)"].median(),
        "std": well_data["Pressure (atm)"].std()
    })

# Преобразуем в DataFrame
pressure_stats_df = pd.DataFrame(pressure_stats)

# Визуализация статистик
plt.figure(figsize=(12, 6))
sns.boxplot(data=pressure_stats_df)
plt.title("Pressure Statistics Across All Wells")
plt.show()

In [ ]:
missing_data = {}
for well_id in file_list:
    well_data = load_well_data(os.path.join(data_path, well_id))
    missing_data[well_id] = well_data.isnull().sum().sum()

# Преобразуем в DataFrame
missing_data_df = pd.DataFrame(list(missing_data.items()), columns=["Well ID", "Missing Values"])

# Визуализация
plt.figure(figsize=(10, 6))
sns.histplot(missing_data_df["Missing Values"], bins=50, kde=True)
plt.title("Distribution of Missing Values Across Wells")
plt.xlabel("Missing Values")
plt.ylabel("Frequency")
plt.show()

In [ ]:
import seaborn as sns
# Визуализация выбросов через boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(x=pressure_stats_df["std"])
plt.title("Boxplot of Pressure Standard Deviation Across Wells")
plt.show()

In [ ]:
import pandas as pd

# Загрузка данных для одной скважины
def load_well_data(file_path):
    df = pd.read_csv(file_path, sep="\t", header=None, names=["Time (hours)", "Pressure (atm)"])
    return df

# Пример загрузки данных для одной скважины
well_id = "0a1c3899-24d5-43aa-9b86-3115b3eeca64"  # Замените на реальный ID скважины
well_data = load_well_data(os.path.join(data_path, well_id))

# Преобразование данных в формат, ожидаемый функцией process_pressure_data
raw_data = list(zip(well_data["Time (hours)"], well_data["Pressure (atm)"]))

In [ ]:
def process_pressure_data(data, t_min, t_max):
    # 1. Фильтрация данных по заданному интервалу времени
    data_filtered = [point for point in data if t_min <= point[0] <= t_max]
    times = np.array([point[0] for point in data_filtered])
    pressures = np.array([point[1] for point in data_filtered])

    # 2. Вычисление модуля ΔP и Δt
    p0 = pressures[0]
    t0 = times[0]
    delta_p = np.abs(pressures - p0)
    delta_t = np.abs(times - t0)

    # 3. Вычисление производной ΔP по логарифму времени (центральная разность)
    p_derivative = np.zeros_like(delta_p, dtype=float)
    for i in range(1, len(delta_t) - 1):
        t_prev, t_next = delta_t[i - 1], delta_t[i + 1]
        dp_prev, dp_next = delta_p[i - 1], delta_p[i + 1]
        p_derivative[i] = (dp_next - dp_prev) / np.log(t_next / t_prev)

    # Удаление «краевых» точек производной (там не определяется центральная разность)
    times_derivative = delta_t[1:-1]
    p_derivative = p_derivative[1:-1]

    # 4. Построение графика в двойных логарифмических координатах
    fig, ax = plt.subplots()
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel("Time (hours)")
    ax.set_ylabel("ΔP and P'")

    # Отображение точек без соединяющей линии
    ax.scatter(delta_t, delta_p, marker='o', label="$ΔP$")
    ax.scatter(times_derivative, p_derivative, marker='s', label="$P'$")

    ax.legend()
    ax.grid(True, which="both", linestyle="--", linewidth=0.5)

    # Установка одинаковой цены деления
    ax.set_aspect('equal', adjustable='datalim')

    plt.show()

# Пример вызова функции для одной скважины
process_pressure_data(raw_data, t_min=50, t_max=150)

## Анализ одной скважины из ground_truth

In [ ]:

def load_well_data(file_path):
    """Загружает данные для одной скважины."""
    df = pd.read_csv(file_path, sep="\t", header=None, names=["Time (hours)", "Pressure (atm)"])
    return df

def process_pressure_data(data, t_min, t_max, interval_type, interval_num):
    """Строит диагностический график для заданного интервала."""
    # Фильтрация данных по заданному интервалу времени
    data_filtered = [point for point in data if t_min <= point[0] <= t_max]
    times = np.array([point[0] for point in data_filtered])
    pressures = np.array([point[1] for point in data_filtered])

    # Вычисление модуля ΔP и Δt
    p0 = pressures[0]
    t0 = times[0]
    delta_p = np.abs(pressures - p0)
    delta_t = np.abs(times - t0)

    # Вычисление производной ΔP по логарифму времени (центральная разность)
    p_derivative = np.zeros_like(delta_p, dtype=float)
    for i in range(1, len(delta_t) - 1):
        t_prev, t_next = delta_t[i - 1], delta_t[i + 1]
        dp_prev, dp_next = delta_p[i - 1], delta_p[i + 1]
        p_derivative[i] = (dp_next - dp_prev) / np.log(t_next / t_prev)

    # Удаление «краевых» точек производной
    times_derivative = delta_t[1:-1]
    p_derivative = p_derivative[1:-1]

    # Построение графика в двойных логарифмических координатах
    fig, ax = plt.subplots()
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel("Time (hours)")
    ax.set_ylabel("ΔP and P'")

    # Отображение точек без соединяющей линии
    ax.scatter(delta_t, delta_p, marker='o', label="$ΔP$")
    ax.scatter(times_derivative, p_derivative, marker='s', label="$P'$")

    ax.legend()
    ax.grid(True, which="both", linestyle="--", linewidth=0.5)

    # Установка одинаковой цены деления
    ax.set_aspect('equal', adjustable='datalim')

    plt.title(f"Diagnostic Plot for {interval_type} Interval {interval_num}")
    plt.show()

def plot_pressure(data, title="Pressure over Time"):
    """Строит общий график давления от времени."""
    plt.figure(figsize=(12, 6))
    plt.scatter(data["Time (hours)"], data["Pressure (atm)"], marker='o', label="Pressure")  # Используем точки
    plt.xlabel("Time (hours)")
    plt.ylabel("Pressure (atm)")
    plt.title(title)
    plt.legend()
    plt.grid()
    plt.show()

def analyze_well(well_id, ground_truth, data_path):
    """Анализирует данные для одной скважины."""
    # Загрузка данных для скважины
    well_data = load_well_data(os.path.join(data_path, well_id))
    raw_data = list(zip(well_data["Time (hours)"], well_data["Pressure (atm)"]))

    # 1. Построение общего графика давления для всего файла
    plot_pressure(well_data, title=f"Pressure over Time for Well {well_id}")

    # Поиск интервалов для данной скважины в ground_truth
    well_intervals = ground_truth[ground_truth["file"] == well_id]
    if well_intervals.empty:
        print(f"No intervals found for well {well_id}")
        return

    # Извлечение интервалов recovery и drop
    recovery_intervals = well_intervals["recovery"].iloc[0]
    drop_intervals = well_intervals["drop"].iloc[0]

    # 2. Построение графиков для интервалов drop
    for i, (start, end) in enumerate(drop_intervals):
        # Общий график для интервала drop
        plot_pressure(well_data[(well_data["Time (hours)"] >= start) & (well_data["Time (hours)"] <= end)],
                     title=f"Drop Interval {i+1} for Well {well_id}")

        # Диагностический график для интервала drop
        process_pressure_data(raw_data, t_min=start, t_max=end, interval_type="Drop", interval_num=i+1)

    # 3. Построение графиков для интервалов recovery
    for i, (start, end) in enumerate(recovery_intervals):
        # Общий график для интервала recovery
        plot_pressure(well_data[(well_data["Time (hours)"] >= start) & (well_data["Time (hours)"] <= end)],
                     title=f"Recovery Interval {i+1} for Well {well_id}")

        # Диагностический график для интервала recovery
        process_pressure_data(raw_data, t_min=start, t_max=end, interval_type="Recovery", interval_num=i+1)

# Пример использования
data_path = "../data/raw/Task 21"
ground_truth = pd.read_csv("../data/raw/ground_truth.csv")
ground_truth["recovery"] = ground_truth["recovery"].apply(ast.literal_eval)  # Преобразуем строки в списки
ground_truth["drop"] = ground_truth["drop"].apply(ast.literal_eval)  # Преобразуем строки в списки


In [ ]:
# Вызов функции для анализа конкретной скважины
analyze_well("00e03657-8e1e-4c8c-a724-1d3c77b48510", ground_truth, data_path)

## Анализ ground_truth

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast  # Для преобразования строк в списки

# Загрузка данных
ground_truth_path = "../data/raw/ground_truth.csv"
ground_truth = pd.read_csv(ground_truth_path)

# Преобразуем строки в списки
ground_truth["recovery"] = ground_truth["recovery"].apply(ast.literal_eval)
ground_truth["drop"] = ground_truth["drop"].apply(ast.literal_eval)

# 1. Анализ структуры данных
print("Общая информация о ground_truth:")
print(ground_truth.info())
print("\nПервые 5 строк:")
print(ground_truth.head())

# 2. Количество скважин и интервалов
num_wells = ground_truth["file"].nunique()
num_recovery_intervals = ground_truth["recovery"].apply(len).sum()
num_drop_intervals = ground_truth["drop"].apply(len).sum()

print(f"\nКоличество скважин: {num_wells}")
print(f"Общее количество интервалов recovery: {num_recovery_intervals}")
print(f"Общее количество интервалов drop: {num_drop_intervals}")

# 3. Анализ длительности интервалов
def calculate_interval_durations(intervals):
    """Вычисляет длительности интервалов."""
    durations = [end - start for interval in intervals for start, end in interval]
    return durations

recovery_durations = calculate_interval_durations(ground_truth["recovery"])
drop_durations = calculate_interval_durations(ground_truth["drop"])

print("\nСтатистика по длительностям интервалов:")
print(f"Recovery: Средняя длительность = {np.mean(recovery_durations):.2f} часов, "
      f"Минимум = {np.min(recovery_durations):.2f} часов, "
      f"Максимум = {np.max(recovery_durations):.2f} часов")
print(f"Drop: Средняя длительность = {np.mean(drop_durations):.2f} часов, "
      f"Минимум = {np.min(drop_durations):.2f} часов, "
      f"Максимум = {np.max(drop_durations):.2f} часов")

# 4. Анализ количества интервалов на скважину
recovery_counts = ground_truth["recovery"].apply(len)
drop_counts = ground_truth["drop"].apply(len)

print("\nСреднее количество интервалов на скважину:")
print(f"Recovery: {recovery_counts.mean():.2f} интервалов на скважину")
print(f"Drop: {drop_counts.mean():.2f} интервалов на скважину")

# 5. Визуализация распределения длительностей
plt.figure(figsize=(12, 6))

# Гистограмма длительностей recovery
plt.subplot(1, 2, 1)
plt.hist(recovery_durations, bins=50, color='blue', alpha=0.7)
plt.title("Распределение длительностей Recovery")
plt.xlabel("Длительность (часы)")
plt.ylabel("Частота")

# Гистограмма длительностей drop
plt.subplot(1, 2, 2)
plt.hist(drop_durations, bins=50, color='red', alpha=0.7)
plt.title("Распределение длительностей Drop")
plt.xlabel("Длительность (часы)")
plt.ylabel("Частота")

plt.tight_layout()
plt.show()

# Boxplot для длительностей
plt.figure(figsize=(10, 6))
plt.boxplot([recovery_durations, drop_durations], labels=["Recovery", "Drop"])
plt.title("Boxplot длительностей интервалов")
plt.ylabel("Длительность (часы)")
plt.show()

# 6. Проверка на пропуски и аномалии
print("\nПроверка на пропуски:")
print(ground_truth.isnull().sum())

# Проверка на нулевые интервалы
zero_recovery = ground_truth[ground_truth["recovery"].apply(len) == 0]
zero_drop = ground_truth[ground_truth["drop"].apply(len) == 0]

print(f"\nКоличество скважин без интервалов recovery: {len(zero_recovery)}")
print(f"Количество скважин без интервалов drop: {len(zero_drop)}")